In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [2]:
import modulos as md
import numpy as np
from backtesting import Backtest
import pandas_ta as ta
from backtesting import Strategy
import pandas as pd
import sys
import os

def backtesting(data, plot_flag=False):
    balance = 100    
    def indicador(df_campo):
        indi=pd.Series(df_campo)
        return indi.to_numpy()
    def ema(df_campo):
        indi=ta.ema(df_campo.s,length=200)
        return indi.to_numpy()   
    class Fenix(Strategy):
        def init(self):
            super().init()
            #self.pivot_high = self.I(indicador,self.data.pivot_high)
            self.pivot_low = self.I(indicador,self.data.pivot_low)
            #self.decisional = self.I(indicador,self.data.decisional)
            self.sma = self.I(ema,self.data.Close)
            #self.tendencia = self.I(indicador,self.data.tendencia)
        def next(self):       
            super().next()
            if self.position:
                if self.data.cierra[-1]==True:
                    self.position.close()                    
            else:   
                if np.isnan(data.take_profit[-1]):
                    tp_value = None
                else:
                    tp_value = self.data.take_profit[-1]
                size= balance*self.data.porcentajeentrada[-1]/100
                if self.data.signal[-1]==1:
                    self.buy(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
                elif self.data.signal[-1]==-1:
                    self.sell(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
    bt = Backtest(data, Fenix, cash=balance)
    output = bt.run()
    if plot_flag:
        bt.plot()
    return output

####################################################################################

def smart_money(df):
    try:
        # Parámetros de longitud para los puntos pivote High y Low
        leftLenH    = 3
        rightLenH   = 3
        leftLenL    = 3
        rightLenL   = 3
        df['pivot_high'] = np.NaN
        df['pivot_low'] = np.NaN
        df['row_number'] = (range(len(df)))
        df.set_index('row_number', inplace=True)
        ## PIVOTS SUPERIORES
        for i in range(leftLenH, len(df) - rightLenH):
            if (
                df['High'].iloc[i] == df['High'].iloc[i - leftLenH:i + rightLenH + 1].max()
                and df['High'].iloc[i] > df['High'].iloc[i - 1]
                and df['High'].iloc[i] > df['High'].iloc[i + 1]
                ):
                df.at[i, 'pivot_high'] = df['High'].iloc[i]
        ## PIVOTS INFERIORES
        for i in range(leftLenL, len(df) - rightLenL):
            if (
                df['Low'].iloc[i] == df['Low'].iloc[i - leftLenL:i + rightLenL + 1].min()
                and df['Low'].iloc[i] < df['Low'].iloc[i - 1]
                and df['Low'].iloc[i] < df['Low'].iloc[i + 1]
                ):
                df.at[i, 'pivot_low'] = df['Low'].iloc[i]
        ## RELLENO DE PIVOTS
        for i in range(0, len(df)):
            if np.isnan(df['pivot_low'].iloc[i]):
                df.at[i, 'pivot_low'] = df['pivot_low'].iloc[i - 1]
            if np.isnan(df['pivot_high'].iloc[i]):
                df.at[i, 'pivot_high'] = df['pivot_high'].iloc[i - 1]   
        #####################
        # DECISIONALES
        df['color'] = np.where(df.Close > df.Open,'verde','rojo')
        df['decisional'] = np.where(                                
                                    (df.color == 'verde') # vela actual verde
                                    & (df.color.shift(1) == 'rojo') # vela anterior roja
                                    & (df.color.shift(-2) == 'rojo') # segunda vela roja                                
                                    & (df.High.shift(-3) < df.Low)    
                                    & (df.atr >= df.Close-df.Open)
                                    ,df.Low,
                                    np.where(
                                            (df.color == 'rojo') 
                                            & (df.color.shift(1) == 'verde') 
                                            & (df.color.shift(-2) == 'verde')
                                            & (df.Low.shift(-3) > df.High)                
                                            & (df.atr >= df.Open-df.Close)
                                            ,df.High,
                                            np.NaN)
                                    )
        df['decisional_flag'] = df['decisional']
        for i in range(0, len(df)):
            if np.isnan(df['decisional'].iloc[i]):
                df.at[i, 'decisional'] = df['decisional'].iloc[i - 1]

        ################# IMBALANCES
        df4h = md.obtiene_historial(symbol,'4h')
        df4h['row_number'] = (range(len(df4h)))
        df4h.set_index('row_number', inplace=True)
        df4h['imba_bajista_high'] = np.where(
                                        ((df4h.Low.shift(1)) >= (df4h.High.shift(-1)+df4h.atr))                  
                                        ,df4h.High,np.NaN)
        df4h['imba_bajista_low'] = np.where(np.isnan(df4h['imba_bajista_high']),np.NaN,df4h.Low)
        df4h['imba_alcista_high'] = np.where(
                                        ((df4h.High.shift(1)) <= (df4h.Low.shift(-1)-df4h.atr))
                                        ,df4h.High,np.NaN)
        df4h['imba_alcista_low'] = np.where(np.isnan(df4h['imba_alcista_high']),np.NaN,df4h.Low)
            ## RELLENO
        for i in range(0, len(df4h)):
            if np.isnan(df4h['imba_bajista_high'].iloc[i]):
                df4h.at[i, 'imba_bajista_high'] = df4h['imba_bajista_high'].iloc[i - 1]
                df4h.at[i, 'imba_bajista_low'] = df4h['imba_bajista_low'].iloc[i - 1]
            if np.isnan(df4h['imba_alcista_high'].iloc[i]):
                df4h.at[i, 'imba_alcista_high'] = df4h['imba_alcista_high'].iloc[i - 1]    
                df4h.at[i, 'imba_alcista_low'] = df4h['imba_alcista_low'].iloc[i - 1]    
        df4h['timestamp']=df4h['Open Time']
        df4h.set_index('timestamp', inplace=True)    
        df4h=df4h[['imba_bajista_high','imba_bajista_low','imba_alcista_high','imba_alcista_low','Open Time']]
        df=pd.merge(df,df4h, on=["Open Time"], how='left')        
        ## indice
        df['timestamp']=df['Open Time']
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass
##########################################################################################

def estrategia_royal(symbol,debug = False):
    try:                
        np.seterr(divide='ignore', invalid='ignore')
        # temporalidad de 1h para encontrar martillos y soportes/resistencias
        data = md.obtiene_historial(symbol,'15m')
        data=smart_money(data)          
        data.drop(['ema20','ema50', 'ema200'], axis=1, inplace=True)    
        data['signal'] = 0
        data['take_profit'] = np.NaN
        data['stop_loss'] = np.NaN
        data['cierra'] = False
        # Reemplazar valores no finitos (NA e inf) con 0
        data['porcentajeentrada'] = np.nan_to_num((data.Close/data.atr), nan=0, posinf=0, neginf=0)
        # Aplicar np.floor y convertir a enteros
        data['porcentajeentrada'] = np.floor(data['porcentajeentrada']).astype(int)
        ####################### alertas y valores
        if debug:
            df_str = data[list(data.columns)].to_string(index=False)
            print(df_str)
        return data
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass

#######################################################################################################################

symbol='BTCUSDT'
data = estrategia_royal(symbol)
backtesting(data, plot_flag=True)

#for index, values in data.iterrows():
#    if not np.isnan(values.decisional_flag):
#        print(f"{values['Open Time']} | {values.Close} | {values.pivot_low} | {values.decisional_flag}")

data.tail(60)     

C:\Users\Luis Gomez Freites\AppData\Local\Temp\ipykernel_14788\2391266812.py:41: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days

,Open Time,Open,High,Low,Close,Volume,atr,pivot_high,pivot_low,color,...,decisional_flag,imba_bajista_high,imba_bajista_low,imba_alcista_high,imba_alcista_low,signal,take_profit,stop_loss,cierra,porcentajeentrada
2023-09-19 06:00:00,2023-09-19 06:00:00,26782.9,26783.0,26740.0,26772.8,1803.737,42.865008,26851.0,26740.0,rojo,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,624
2023-09-19 06:15:00,2023-09-19 06:15:00,26772.8,26784.7,26752.6,26762.5,987.838,42.096079,26851.0,26740.0,rojo,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,635
2023-09-19 06:30:00,2023-09-19 06:30:00,26762.5,26820.9,26757.9,26810.0,1211.676,43.589216,26851.0,26740.0,verde,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,615
2023-09-19 06:45:00,2023-09-19 06:45:00,26810.1,26817.0,26796.1,26816.9,857.249,41.968558,26851.0,26740.0,verde,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,638
2023-09-19 07:00:00,2023-09-19 07:00:00,26816.9,26819.6,26797.5,26817.1,757.176,40.549375,26851.0,26740.0,verde,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,661
2023-09-19 07:15:00,2023-09-19 07:15:00,26817.0,26847.6,26809.8,26841.9,1249.700,40.352991,26851.0,26740.0,verde,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,665
2023-09-19 07:30:00,2023-09-19 07:30:00,26841.9,26857.6,26821.6,26831.7,1355.447,40.042063,26851.0,26740.0,rojo,...,26857.6,NaN,NaN,NaN,NaN,0,NaN,NaN,False,670
2023-09-19 07:45:00,2023-09-19 07:45:00,26831.7,26860.0,26826.0,26836.1,1603.038,39.610487,26851.0,26740.0,verde,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,677
2023-09-19 08:00:00,2023-09-19 08:00:00,26836.0,26889.7,26834.3,26889.6,3864.402,40.738309,26851.0,26740.0,verde,...,NaN,25749.7,24915.0,27235.0,26666.4,0,NaN,NaN,False,660
2023-09-19 08:15:00,2023-09-19 08:15:00,26889.6,26930.0,26870.2,26925.1,3915.894,42.099859,26851.0,26740.0,verde,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,False,639
